In [ ]:
!pip install -q torch==2.1.0 --index-url https://download.pytorch.org/whl/cu121

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 981.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 18.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.1.0+cu121 which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.1.0+cu121 which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.1.0+cu121 which is incompatible.


In [ ]:
!pip install -q "transformers==4.34.0" "datasets==2.13.0" "peft==0.6.2" "accelerate==0.21.0" "bitsandbytes==0.41.2.post2" "trl==0.4.7" "safetensors>=0.3.1" "gradio==4.8.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 12.4 MB/s eta 0:00:00
  Preparing metadat

In [ ]:
import pandas as pd
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import PeftModel, PeftConfig, get_peft_model
from transformers import pipeline
from datetime import datetime, timedelta
from huggingface_hub import login
from google.colab import userdata

In [ ]:
login(token=userdata.get('niru_hf_read'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
def get_pipline():
    # model_name = "nirusanan/Mistral_7B_SFT_Finetune_Pandas_Code_Gen_Final"
    model_name = "nirusanan/Mistral_7B_DPO_Finetune_Pandas_Code_Gen_Final"
    # model_name = "nirusanan/mistral-finetuned-4bit-v10"
    # model_name = "chelvan/Mistral_7B_SFT_Finetune_Pandas_Code_Gen_V2"

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="cuda:0",
        trust_remote_code=True
    )

    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=850)

    return pipe

pipe = get_pipline()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
def generate_prompt(task, header_columns):
    prompt = f"""Below is an instruction that describes a task. Write a Python function using Pandas to accomplish the task described below.

### Instruction:
{task}

header columns with sample data:
{header_columns}

### Response:
"""
    return prompt

In [ ]:
def format_dataframe(df):
    df_dict = df.head(5).to_dict(orient='list')
    formatted_string = "data = {\n"

    for key, values in df_dict.items():
        formatted_string += "    '{}': {},\n".format(key, values)

    formatted_string += "}"
    return formatted_string

In [ ]:
import re

def generate_code(prompt):
    result = pipe(prompt)
    generated_text = result[0]['generated_text']
    res = generated_text.split("### End")[0]

    # Search for the Python code block
    match = re.search("```(.*?)```", res, re.DOTALL)

    # Check if a match was found
    if match:
        python_code = match.group(1).strip()
    else:
        # If no match was found, handle the error accordingly
        python_code = generated_text

    return python_code

In [ ]:
import traceback

def filter_dataframe(df, query):
    # Initialize python_code to ensure it has a value even if the subsequent code fails
    python_code = ""

    try:
        header_column = format_dataframe(df)
        prompt = generate_prompt(query, header_column)
        python_code = generate_code(prompt)

        python_code = "global process\n" + python_code
        exec(python_code)
        results = process(df)

        # # If the process function was defined in the executed code, it can be called
        # if 'process' in local_namespace:
        #     results = local_namespace['process'](df)
        # else:
        #     raise NameError("No 'process' function found in the executed code.")

        # # Ensure results is in the correct format for return (e.g., if it's a DataFrame, convert to JSON)
        # if hasattr(results, 'to_json'):
        #     results = results.to_json()
        # else:
        #     results = str(results)

        return python_code, results

    except Exception as e:
        # Capture and format the full traceback
        full_traceback = traceback.format_exc()

        # Provide both the error message and the traceback for detailed debugging information
        error_message = f"An error occurred: {e}\n{full_traceback}"

        # Include the python_code variable in the message if it has been defined
        final_msg = f"{python_code}\n======Error======\n{error_message}"

        return final_msg, "Error"

In [ ]:
import gradio as gr
import pandas as pd

# Global variable to store the full DataFrame
full_df = None

# Function to process the uploaded CSV and show the first 5 records.
def process_csv(uploaded_file):
    global full_df
    if uploaded_file is None:
        return "Please upload a CSV file."
    full_df = pd.read_csv(uploaded_file)
    return full_df.head()

def handle_query(query):
    global full_df
    if full_df is None:
        return "Please upload a CSV file to enable query processing.", None
    if query == "":
        return "Please enter a query to generate Python code and results.", None

    python_code, results = filter_dataframe(full_df, query)

    return python_code, results

# Create the Gradio app.
with gr.Blocks() as app:
    gr.Markdown("## Upload CSV and Query Data")

    # File upload component.
    with gr.Column():
        file_input = gr.File(label="Choose a CSV file")

    # Sample records display
    with gr.Column():
        sample_display = gr.Dataframe(label="Sample Records")

    # User query input
    with gr.Column():
        query_input = gr.Textbox(label="Enter your query (e.g., 'Filter rows where `sales` > 1000'):")

    # Run Query button
    with gr.Column():
        run_button = gr.Button("Run Query")

    # Textbox for Python code generation.
    with gr.Column():
        code_output = gr.Textbox(label="Generated Python Code", value="Your code will be generated here.")

    # Placeholder for query results.
    with gr.Column():
        query_results = gr.Textbox(label="Query Results", value="Results of your query will appear here.")

    # Interaction logic.
    file_input.change(fn=process_csv, inputs=file_input, outputs=sample_display)
    run_button.click(fn=handle_query, inputs=query_input, outputs=[code_output, query_results])

app.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://04070de536156020ac.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
